In [34]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import math
import random

screenWidth = 1200
screenHeight = 700
def setColor(color):
    r, g, b = color
    glColor3f(r/255, g/255, b/255)
def drawSphere(radius):
    quad = gluNewQuadric()
    gluQuadricDrawStyle(quad, GLU_FILL)
    gluSphere(quad, radius, 16, 16)
    glColor3f(0, 0, 0)
    gluQuadricDrawStyle(quad, GLU_LINE)
    gluSphere(quad, radius, 16, 16)
    
def drawStars(stars, twinkle_state):
    glPointSize(4)
    glBegin(GL_POINTS)
    for i, (x, y, z) in enumerate(stars):
        brightness = 255 if twinkle_state[i] else 128
        setColor((brightness, brightness, brightness))
        glVertex3f(x, y, z)
    glEnd()

def drawRing(inner_radius, outer_radius, segments=64):
    glBegin(GL_QUAD_STRIP)
    for i in range(segments + 1):
        angle = 2 * math.pi * i / segments
        x_inner = inner_radius * math.cos(angle)
        y_inner = inner_radius * math.sin(angle)
        x_outer = outer_radius * math.cos(angle)
        y_outer = outer_radius * math.sin(angle)
        glVertex3f(x_inner, y_inner, 0)
        glVertex3f(x_outer, y_outer, 0)
    glEnd()

def main():
    pygame.init()
    display = (screenWidth, screenHeight)
    screen = pygame.display.set_mode(display, DOUBLEBUF|OPENGL)
    pygame.display.set_caption("Solar System 3D")
    clock = pygame.time.Clock()

    pygame.mixer.init()
    move_sound = pygame.mixer.Sound("cru.wav")


    glMatrixMode(GL_PROJECTION)
    gluPerspective(45, (display[0]/display[1]), 0.1, 1000.0)
    glMatrixMode(GL_MODELVIEW)

    myX = 0
    myY = -80
    myZ = 10
    myHeading = 90.0
    myPitch = 0.0

    gluLookAt(myX, myY, myZ, 0, 0, 0, 0, 0, 1)
    viewMatrix = glGetFloatv(GL_MODELVIEW_MATRIX)
    glLoadIdentity()

    stars = [(random.uniform(-300, 300), random.uniform(-300, 300), random.uniform(-300, 300)) for _ in range(600)]
    twinkle_state = [random.choice([True, False]) for _ in range(600)]

    planet_data = [
        (10, 0.5, 4.7, (80, 200, 120), [], 4),
        (15, 0.8, 3.5, (0, 128, 128), [], 3),
        (20, 1.0, 2.9, (70, 130, 180), [(2, 0.2, 5, (255, 215, 0))], 4),
        (25, 0.7, 0.5, (200, 0, 0), [], 2),
        (32, 2.0, 1.3, (255, 0, 255), [(3, 0.3, 7, (0, 0, 255))], 1.5),
        (40, 1.8, 1.0, (150, 123, 182), [(4, 0.3, 5, (255, 255, 255)), (5, 0.4, -5, (200, 0, 0))], 1),
        (48, 1.5, 0.7, (212, 128, 144), [], 1),
        (55, 1.2, 1.5, (128, 0, 128), [], 2)
    ]

    angle = 0
    running = True
    while running:
        glLoadIdentity()
        glPushMatrix()
        glLoadIdentity()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        keys = pygame.key.get_pressed()
        moving = False  
        #  movement key          
        if keys[pygame.K_UP]:
            glTranslatef(0, 0, 0.5)
            myZ += 0.5
            moving = True
        if keys[pygame.K_DOWN]:
            glTranslatef(0, 0, -0.5)
            myZ -= 0.5
            moving = True
        if keys[pygame.K_LEFT]:
            glTranslatef(0.5, 0, 0)
            myX += 0.5
            moving = True
        if keys[pygame.K_RIGHT]:
            glTranslatef(-0.5, 0, 0)
            myX -= 0.5
            moving = True
        if keys[pygame.K_PAGEUP]:
            glTranslatef(0, -0.5, 0)
            myY -= 0.5
            moving = True
        if keys[pygame.K_PAGEDOWN]:
            glTranslatef(0, 0.5, 0)
            myY += 0.5
            moving = True
        
        # rotatekeys  
        if keys[pygame.K_w]:
            glRotatef(-1, 1, 1, 0)
            myHeading += 0.01
            moving = True
        if keys[pygame.K_s]:
            glRotatef(1, 1, 1, 0)
            myHeading -= 0.01
            moving = True
        if keys[pygame.K_d]:
            glRotatef(1, 1, 1, 0)
            myPitch -= 0.01
            moving = True
        if keys[pygame.K_a]:
            glRotatef(-1, 1, 1, 0)
            myPitch += 0.01
            moving = True
        
         
        if moving:
            if not pygame.mixer.Channel(0).get_busy():
                pygame.mixer.Channel(0).play(move_sound, loops=-1)
        else:
            pygame.mixer.Channel(0).stop()


        glMultMatrixf(viewMatrix)
        viewMatrix = glGetFloatv(GL_MODELVIEW_MATRIX)
        glPopMatrix()
        glMultMatrixf(viewMatrix)

        glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)

        glEnable(GL_DEPTH_TEST)
        drawStars(stars, twinkle_state)
        twinkle_state = [not state if random.random() < 0.05 else state for state in twinkle_state]

        setColor((255, 165, 0))
        glPushMatrix()
        drawSphere(5)
        glPopMatrix()

        for j, (distance, size, speed, color, moons, self_spin) in enumerate(planet_data):
            glPushMatrix()
            glRotatef(angle * speed, 0, 0, 1)
            glTranslatef(distance, 0, 0)
            glPushMatrix()
            glRotatef(angle, 0, 1, 0)
            setColor(color)
            drawSphere(size)
            glPopMatrix()

            if j == 5:
                glDisable(GL_LIGHTING)
                setColor((200, 200, 200))
                glPushMatrix()
                glRotatef(20, 0, 1, 0)
                drawRing(2.3, 2.6)
                drawRing(2.7, 3.0)
                glPopMatrix()

            for moon_distance, moon_size, moon_speed, moon_color in moons:
                glPushMatrix()
                glRotatef(angle * moon_speed, 0, 0, 1)
                glTranslatef(moon_distance, 0, 0)
                setColor(moon_color)
                drawSphere(moon_size)
                glPopMatrix()
            glPopMatrix()

        angle += 0.2




        
        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

main()